In [1]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import uuid

In [2]:
trait_name = "python_client_demo.test"
layer_name = "testlayer"

create a omnikeeper client

In [3]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

ensure trait exists

In [4]:
okc.upsert_trait(okapiclient, TraitDefinition(trait_name, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT, is_array=True),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))

True

ensure layer exists

In [5]:
okc.create_layer(okapiclient, layer_name, "just a test layer", okc.hex_string_to_rgb_color("#6666FF"))

True

Create the input data in format of **input : List[Dict[str,Any]]**

In [6]:
initial_data = [
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
]
initial_data

[{'id': 1, 'array': ['a', 'b'], 'some_key': 'Value 1'},
 {'id': 3, 'array': ['c', 'd'], 'some_key': 'another Value'}]

write data to layer ``testlayer``

In [7]:
okc.bulk_replace_trait_entities_by_filter_list(okapiclient, trait_name=trait_name, input=initial_data, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

True

load data again from omnikeeper

In [8]:
updated_data = okc.get_all_traitentities_list(okapiclient, trait_name=trait_name, layers=[layer_name])
updated_data

[{'ciid': '1991b3a7-68fb-45c8-9349-720eb1e6867c',
  'id': 1,
  'array': ['a', 'b'],
  'some_key': 'Value 1'},
 {'ciid': 'd8040611-e63d-4798-9c6f-46d7b6efa57f',
  'id': 3,
  'array': ['c', 'd'],
  'some_key': 'another Value'}]

modify the fetched traitentities

In [9]:
# change things in the returned data
# change single value of existing data
updated_data[0]["array"] = ["a", "b", "z"]

# add a new row, using our own created CIID
updated_data.append({"ciid": str(uuid.uuid4()), "id": 4, "array": ["x"], "some_key": "the new guy"})

# drop a row
updated_data.pop(1)
updated_data

[{'ciid': '1991b3a7-68fb-45c8-9349-720eb1e6867c',
  'id': 1,
  'array': ['a', 'b', 'z'],
  'some_key': 'Value 1'},
 {'ciid': '16c28c57-60dd-4f0e-83d9-9c773d257fff',
  'id': 4,
  'array': ['x'],
  'some_key': 'the new guy'}]

write it back

In [10]:
okc.bulk_replace_trait_entities_list(okapiclient, trait_name=trait_name, input=updated_data, write_layer=layer_name)

True

Fetch again the data

In [11]:
final_data = okc.get_all_traitentities_list(okapiclient, trait_name=trait_name, layers=[layer_name])
final_data

[{'ciid': '1991b3a7-68fb-45c8-9349-720eb1e6867c',
  'id': 1,
  'array': ['a', 'b', 'z'],
  'some_key': 'Value 1'},
 {'ciid': '16c28c57-60dd-4f0e-83d9-9c773d257fff',
  'id': 4,
  'array': ['x'],
  'some_key': 'the new guy'}]